<a href="https://colab.research.google.com/github/stephenbeckr/numerical-analysis-class/blob/master/Demos/Ch1_QuadraticFormula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Condition numbers and algorithms for the quadratic formula

## Condition number
If $r_1$ and $r_2$ are the roots, then we derived in our lecture that the condition number of finding the root $r$, with respect to perturbations in the coefficient $a$ (using $p(x) = ax^2 + bx + c$) is
$$
\kappa(a) = \left| \frac{r}{r_1-r_2} \right|
$$
so if the absolute value of a root $|r|$ is much greater than the spacing between the roots $|r_1-r_2|$, then root finding becomes ill-conditioned.

Below is an example



In [33]:
import math

def myRoots( a, b, c):
  discSq = math.sqrt( b**2 - 4*a*c)
  return (-b + discSq)/(2*a) , (-b - discSq)/(2*a) 

# Make a polynomial
r1 = 1e3 + 1.01
r2 = 1e3 
a , b, c = 1., -(r1+r2), r1*r2

s1, s2 = myRoots( a, b, c)
print( "Error is {:e} and {:e}".format(abs(s1-r1), abs(s2-r2)) )


Error is 3.740297e-11 and 3.740297e-11


The error in our input is about $10^{-16}$ due to floating point, and then we have a condition number of about $10^3$ with respect to the coefficient $a$, so we should lose at least 3 digits, and expect an answer no more accurate than $10^{-13}$. In fact, it's even worse, because we have to take into account error from $b$ and $c$, as well as the implementation of the root-finding is not perfect (more about that shortly!)

## Changing the algorithm
For more accuracy, we can not use the quadratic formula.  Supposing $b>0$, then when we use the formula
$$
r_1 = \frac{ - b + \sqrt{b^2 - 4ac} }{2a}
$$
then if $b^2 \gg 4ac$, we have $\sqrt{b^2 - 4ac} \approx b$, and so we'll have **subtractive cancellation** and lose precision.

Instead, rationalize the denominator,
$$
r_1 = \frac{ - b + \sqrt{b^2 - 4ac} }{2a} \cdot 
\frac{ - b - \sqrt{b^2 - 4ac} }{- b - \sqrt{b^2 - 4ac}}
= \frac{ b^2 - (b^2 - 4ac)}{ 2a(-b-\sqrt{b^2 - 4ac} )}
$$
and then cancel the $b^2 - b^2$ **by hand** in the numerator (rather than relying on the computer to do it) and simplify to get
$$ r_1 = \frac{-2c}{b+\sqrt{b^2-4ac}}.
$$

Note that if $b<0$, we do **not** want to do this formula, since then we have subtractive cancellation in the denominator, and it's even worse!  In this case, we can use the old formula for $r_1$ but derive a similar new formula for $r_2$


In [31]:
# Now, make a better root function

def myRootsBetter( a, b, c):
  discSq = math.sqrt( b**2 - 4*a*c)
  if b > 0 :
    r1 = -2*c/(b+math.sqrt(b**2-4*a*c))
    r2 = (-b - discSq)/(2*a)
  else :
    r1 = (-b + discSq)/(2*a)
    r2 = -2*c/(b-math.sqrt(b**2-4*a*c))
  return r1, r2

s1, s2 = myRootsBetter( a, b, c)
print( "Error (new method) is {:e} and {:e}".format(abs(s1-r1), abs(s2-r2)) )

Error (new method) is 0.000000e+00 and 0.000000e+00


Let's apply this to a "nice" polynomial (one that according to our analysis should be well-conditioned).  The function is not ill-conditioned, but our **algorithm** could be unstable.

In [38]:
# Now, on a different kind of polynimial
r1 = 1.0e6 + 1.01
r2 = 1.01e-3 
a , b, c = 1., -(r1+r2), r1*r2
print("a, b, c are ", a, b, c )

s1, s2 = myRoots( a, b, c)
print( "Error is {:e} and {:e}".format(abs(s1-r1), abs(s2-r2)) )

s1, s2 = myRootsBetter( a, b, c)
print( "Error (new method) is {:e} and {:e}".format(abs(s1-r1), abs(s2-r2)) )

a, b, c are  1.0 -1000001.01101 1010.0010201
Error is 0.000000e+00 and 5.098060e-11
Error (new method) is 0.000000e+00 and 0.000000e+00
